<a href="https://colab.research.google.com/github/kbcvcbk/cesar-school/blob/master/estatistica-e-probabilidade/nascer-do-sol/Ricardo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Etapa 1: Pré-processamento dos dados.
## Links
- [Atividade no Classroom](https://classroom.google.com/u/0/c/MTQ4MzY4NzYyMDM3/a/MTU1OTk5ODg0OTcx/details)
- [Artigo de Ricardo](https://drive.google.com/file/d/1dNBfmRlWIvUj_8mzpTEBtRqQF9tVl0kh/view)
- [Cálculo de nascer e por do sol](https://www.inf.ufrgs.br/~cabral/Nascer_Por_Sol.html#:~:text=Subtraindo%206%20horas%20do%20meio,pa%C3%ADs%2C%20estes%20hor%C3%A1rios%20seriam%20corretos.)

## Enunciado
Esta etapa consiste na conversão das fórmulas disponíveis no artigo para programação juntamente com o tratamento dos dados.

O tratamento dos dados deverá ter foco no horário do nascimento do sol na cidade de Água Branca - AL, convertendo o tempo total de sol em minutos por dia.

Para isso, você precisa calcular a hora que o sol nasce para um dado dia (Utilizar o artigo Cálculo do Nascer e Pôr do Sol).

A entrega deverá ser feita até o dia 03/11, as 23:59.

## Etapas
1. Conversão das fórmulas do artigo
2. Cálcular a hora que o sol nasce cada dia
3. Converter o tempo total de sol em minutos por dia

In [92]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Ler XLS e montar o DataFrame
Fiz upload do arquivo aqui no colab (vocês podem ver clicando na pasta na que tem lá na esquerda), então ele deve ficar disponível pra todo mundo.

### Refs
- [Função read_excel](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_excel.html)

In [93]:
pag = pd.read_excel("Janeiro 2008.xls",
                    sheet_name = [0, 1, 2],
                    skiprows = [0, 2, 3],
                    header = 0)

df = pd.concat([pag[0], pag[1], pag[2]])

In [94]:
df

,TIMESTAMP,RECORD,CR1000_Bat_Avg,VelVento,DirVento,TempAR_Avg,RH_Max,RadHZtot_Avg,RadPAR_Avg,IlumHZ_Avg,IlumNORTE_Avg,IlumSUL_Avg,IlumLESTE_Avg,IlumOESTE_Avg
0,2008-01-01 00:00:00,243801,13.35,6.147,111.8,20.06,80.4,0.0,-0.011,0.067,18.590,0.067,1.549,1.326
1,2008-01-01 00:01:00,243802,13.35,5.113,111.1,20.07,80.4,0.0,0.022,0.067,18.590,0.067,1.549,1.318
2,2008-01-01 00:02:00,243803,13.35,5.265,107.3,20.04,80.4,0.0,0.045,0.067,18.590,0.067,1.549,1.324
3,2008-01-01 00:03:00,243804,13.35,5.289,105.4,20.04,80.5,0.0,0.011,0.067,18.590,0.067,1.549,1.329
4,2008-01-01 00:04:00,243805,13.35,6.436,105.8,20.04,80.6,0.0,-0.067,0.067,18.590,0.067,1.549,1.340
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15835,2008-01-31 23:55:00,288436,13.31,1.300,130.1,21.54,94.9,0.0,-0.011,0.067,3.637,0.067,1.145,1.347
15836,2008-01-31 23:56:00,288437,13.31,1.522,118.3,21.54,94.8,0.0,-0.045,0.067,3.637,0.067,1.145,1.347
15837,2008-01-31 23:57:00,288438,13.31,1.799,115.0,21.54,94.7,0.0,-0.101,0.067,3.637,0.067,1.145,1.347
15838,2008-01-31 23:58:00,288439,13.31,1.584,122.6,21.55,94.7,0.0,-0.112,0.067,3.637,0.067,1.145,1.360


### Conversão das fórmulas do artigo

In [95]:
from numpy import (
    sin,
    cos,
    tan,
    arccos,
    radians,
    degrees,
)

from datetime import (
    timedelta,
    time,
)

In [96]:
def earth_declination(n):
    return 23.45 * sin(radians(360/365 * (284+n)))

def td(lat):
    dec = earth_declination(119) #TODO Change this literal
    cofactor = -(tan(radians(lat)) * tan(radians(dec)))
    return 2/15 * degrees(arccos(cofactor))

def longitude_correction(lng, fuse):
    diff = lng - fuse
    
    return timedelta(
        minutes=(diff * 60) / 15
    )

def day_range(td, lng, fuse):
    td /= 2
    sunrise = timedelta(hours=12-td)
    sunset = timedelta(hours=12+td)

    correction = longitude_correction(lng, fuse)
    sunrise -= correction
    sunset -= correction
    
    return (sunrise, sunset)

In [97]:
"""
Teste com dados do artigo
valores esperados:
daytime = 11,15174 
sunrise = 6h 25min 26s
sunset = 17h 43min 33s
"""

lat = -23.543333
lng = 46.633056
fuse = 45

daytime = td(lat)
sunrise, sunset = day_range(daytime, lng, fuse)

print(f"""daytime = {daytime}
sunrise = {sunrise}
sunset = {sunset}""")

daytime = 11.151741131164016
sunrise = 6:18:54.932524
sunset = 17:28:01.200596
